# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F

In [4]:
data_dir = "/media/mountHDD2/khoibaocon"
print(os.listdir(data_dir))

['TrainingSet3', 'Label.csv', 'alldata', 'TrainingSet1', 'single_label.csv', 'TrainingSet2']


In [5]:
main_df = pd.read_csv(data_dir + "/Label.csv")
main_df.shape

(6877, 4)

In [6]:
single_main_df = main_df[main_df["Second_label"].isnull()]
single_fns = single_main_df["Recording"].values.tolist()
single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]

In [7]:
ratio = [0.9, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[train_index:]

train_label = single_main_df.iloc[:train_index,:]
valid_label = single_main_df.iloc[train_index:,:]

 # Data Loader

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [9]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        data = loadmat(data_path)['ECG'][0][0][2]
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        grayscale_images = (normalized_data * 255)
        grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        resized_images = F.interpolate(grayscale_images, size=(12*4,2500), mode='bilinear', align_corners=True)
        resized_images = resized_images.squeeze(0).squeeze(0)
        torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["Recording"] == filename]["First_label"].values.item()

        return torch_data_resize, label-1

    def __len__(self):
        return len(self.data_paths)    


In [10]:
train_ds = HeartData(train_mat_paths, single_main_df)
valid_ds = HeartData(valid_mat_paths, single_main_df)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [12]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 9),
            nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        x = self.ori_model(x)
        return x

model = HeartModel()
# x = torch.randn((1, 3, 256, 512)).to("cuda")
# out = model(x)
# print(out.shape)

In [13]:
model

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [14]:
epoch = 300
lr = 0.001 # lr = 0.001 Acc: 82.2 lr = 0.0005 Acc: 0.822
best_acc = 0
best_ep = 0

model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))
loss_fn = nn.CrossEntropyLoss()

In [15]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=1):
        super(FocalClassifierV0, self).__init__()
        
        self.gamma = gamma
    
    def forward(self, pred, target) -> torch.Tensor:

        logits = F.log_softmax(pred, dim=1)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits.exp(), self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        loss = (-1) * entropy.mean()

        return loss
    
focalloss_fn = FocalClassifierV0()

In [16]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

Epoch: 0


90it [00:10,  8.44it/s]

train loss: 0.17188771320193003 - train acc: 56.21527777777777



640it [00:04, 150.76it/s]

valid loss: 1.8455065589555553 - valid acc: 52.65625000000001
Epoch: 1


...

valid loss: 1.5722668866782867 - valid acc: 80.0
Epoch: 299



90it [00:08, 10.93it/s]

train loss: 0.1196042447445098 - train acc: 96.40625



640it [00:04, 152.59it/s]

valid loss: 1.5711711551848337 - valid acc: 80.3125
Best acuracy: 0.821875 at epoch 166


# Evaluation